In [1]:
%pylab inline
import pandas as pd
import workalendar.europe.belgium as belgium
import sklearn
import tqdm

Populating the interactive namespace from numpy and matplotlib


In [2]:
raw_data = pd.read_csv("./data/dataraw.csv", parse_dates=["time"])

In [3]:
cleaned_data = raw_data[["time", "is_open"]]
cleaned_data.loc[cleaned_data.time.dt.hour >= 23, "is_open"] = False
cleaned_data.loc[cleaned_data.time.dt.hour < 7, "is_open"] = False
cleaned_data["date"] = cleaned_data.time.dt.date

cleaned_data.sample(10)

/home/nikita/Code/openings/ve/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,time,is_open,date
15152,2014-08-30 22:00:00,False,2014-08-30
17235,2014-11-25 17:00:00,True,2014-11-25
27708,2016-02-05 02:00:00,False,2016-02-05
22388,2015-06-28 10:00:00,False,2015-06-28
12422,2014-05-09 04:00:00,False,2014-05-09
36954,2017-02-24 08:00:00,False,2017-02-24
922,2013-01-15 00:00:00,False,2013-01-15
18738,2015-01-27 08:00:00,False,2015-01-27
26015,2015-11-26 13:00:00,False,2015-11-26
9234,2013-12-27 08:00:00,False,2013-12-27


In [4]:
data_dates = pd.Series(cleaned_data["date"].unique(), name="day")
data_dates.sample(10)

205     2013-06-30
1422    2016-10-29
1611    2017-05-06
220     2013-07-15
572     2014-07-02
928     2015-06-23
1639    2017-06-03
21      2012-12-28
1072    2015-11-14
146     2013-05-02
Name: day, dtype: object

In [5]:
starting_days = [
    datetime.date(2009, 9, 14),
    datetime.date(2010, 9, 20),
    datetime.date(2011, 9, 19),
    datetime.date(2012, 9, 17),
    datetime.date(2013, 9, 16),
    datetime.date(2014, 9, 15),
    datetime.date(2015, 9, 14),
    datetime.date(2016, 9, 19),
    datetime.date(2017, 9, 18),
]

assert all([d.weekday() == 0 for d in starting_days])

In [6]:
def get_week_no(date):
    possible = [d for d in starting_days if d <= date]
    aca_start = possible[-1]
    dt = date - aca_start
    return math.floor(timedelta64(dt) / timedelta64(1,'W'))

week_df = data_dates.to_frame()
week_df["week_no"] = week_df["day"].apply(get_week_no)
week_df["day"] = pd.to_datetime(week_df["day"])

week_df[week_df.day.dt.date > datetime64("2013-09-10")].head(15)

,day,week_no
278,2013-09-11,51
279,2013-09-12,51
280,2013-09-13,51
281,2013-09-14,51
282,2013-09-15,51
283,2013-09-16,0
284,2013-09-17,0
285,2013-09-18,0
286,2013-09-19,0
287,2013-09-20,0


In [7]:
featurized = cleaned_data.copy()

# day of the week
featurized["weekday"] = featurized["time"].dt.weekday

# academic week number
featurized["aca_week_no"] = featurized["date"].apply(get_week_no)

# civil week number
featurized["week_no"] = featurized["time"].dt.week

# date
featurized["year"] = featurized["time"].dt.year
featurized["month"] = featurized["time"].dt.month
featurized["day"] = featurized["time"].dt.day
featurized["hour"] = featurized["time"].dt.hour

# holiday
calendar = belgium.Belgium()
featurized["working"] = featurized["date"].apply(calendar.is_working_day)
### St V
featurized.loc[(featurized["time"].dt.month == 11) & (featurized["time"].dt.day == 20), "working"] = False

# tampon
featurized["tampon"] = False
featurized.loc[featurized.aca_week_no.isin([6, 13, 26, 34]), "tampon"] = True

# exams
featurized["exams"] = False
featurized.loc[featurized.aca_week_no.isin([14, 15, 16, 17, 18]), "exams"] = True
featurized.loc[featurized.aca_week_no.isin([36, 37, 38, 39, 40]), "exams"] = True

# vacances
featurized["vacances"] = False
featurized.loc[featurized.aca_week_no.isin([19] + list(range(41, 52))), "vacances"] = True

# weather

featurized.sort_values("time")
featurized.sample(5)

,time,is_open,date,weekday,aca_week_no,week_no,year,month,day,hour,working,tampon,exams,vacances
17576,2014-12-09 22:00:00,False,2014-12-09,1,12,50,2014,12,9,22,True,False,False,False
30279,2016-05-22 05:00:00,False,2016-05-22,6,35,20,2016,5,22,5,False,False,False,False
32054,2016-08-04 04:00:00,False,2016-08-04,3,46,31,2016,8,4,4,True,False,False,True
33318,2016-09-25 20:00:00,False,2016-09-25,6,0,38,2016,9,25,20,False,False,False,False
18201,2015-01-04 23:00:00,False,2015-01-04,6,15,1,2015,1,4,23,False,False,True,False


In [8]:
xcols = list(set(featurized.columns) - set(["is_open", "date", "time"]))
one_hot = [
    "weekday",
    "aca_week_no", 
    "week_no",
    
    "year",
    "month",
    "day",
    "hour",
]

I = featurized["time"].values
X = pd.get_dummies(featurized[xcols], columns=one_hot).values.astype(bool)
Y = featurized["is_open"].values

In [9]:
X.shape

(39965, 190)

In [10]:
Y.shape

(39965,)

In [11]:
# List of features
xcols

['exams',
 'working',
 'hour',
 'month',
 'week_no',
 'vacances',
 'weekday',
 'tampon',
 'aca_week_no',
 'year',
 'day']

In [12]:
# List of one-hotted features
features = pd.get_dummies(featurized[xcols], columns=one_hot).columns.values
features

array(['exams', 'working', 'vacances', 'tampon', 'weekday_0', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'aca_week_no_0', 'aca_week_no_1', 'aca_week_no_2', 'aca_week_no_3',
       'aca_week_no_4', 'aca_week_no_5', 'aca_week_no_6', 'aca_week_no_7',
       'aca_week_no_8', 'aca_week_no_9', 'aca_week_no_10',
       'aca_week_no_11', 'aca_week_no_12', 'aca_week_no_13',
       'aca_week_no_14', 'aca_week_no_15', 'aca_week_no_16',
       'aca_week_no_17', 'aca_week_no_18', 'aca_week_no_19',
       'aca_week_no_20', 'aca_week_no_21', 'aca_week_no_22',
       'aca_week_no_23', 'aca_week_no_24', 'aca_week_no_25',
       'aca_week_no_26', 'aca_week_no_27', 'aca_week_no_28',
       'aca_week_no_29', 'aca_week_no_30', 'aca_week_no_31',
       'aca_week_no_32', 'aca_week_no_33', 'aca_week_no_34',
       'aca_week_no_35', 'aca_week_no_36', 'aca_week_no_37',
       'aca_week_no_38', 'aca_week_no_39', 'aca_week_no_40',
       'aca_week_no_41', 'aca_week

In [13]:
list(features).index("year_2017")

122

In [14]:
test_mask = X[:,122]
train_mask = np.invert(test_mask)

Xtrain = X[train_mask]
Ytrain = Y[train_mask]

Xtest = X[test_mask]
Ytest = Y[test_mask]

print(Xtrain.shape, Ytrain.shape)
print(Xtest.shape, Ytest.shape)

(35650, 190) (35650,)
(4315, 190) (4315,)


In [15]:
import sklearn.ensemble

In [16]:
ada = sklearn.ensemble.AdaBoostClassifier()
ada.fit(Xtrain, Ytrain)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [17]:
Yada = ada.predict(Xtest)

In [18]:
sklearn.metrics.matthews_corrcoef(Ytest, Yada)

0.57227465142780976

In [19]:
import sklearn.neighbors
knn = sklearn.neighbors.KNeighborsClassifier(20)
knn.fit(Xtrain, Ytrain)
Yknn = knn.predict(Xtest)

In [20]:
sklearn.metrics.matthews_corrcoef(Ytest, Yknn)

0.19378445666356187

In [21]:
forest = sklearn.ensemble.RandomForestClassifier()
forest.fit(Xtrain, Ytrain)
Yforest = forest.predict(Xtest)

In [22]:
sklearn.metrics.matthews_corrcoef(Ytest, Yforest)

0.53704793106065185

# Cross validation

In [23]:
print(I.shape, X.shape, Y.shape)

(39965,) (39965, 190) (39965,)


In [51]:
def balance(X, Y):
    t_count = Y.sum()
    f_count = Y.shape[0] - t_count
    
    c_max = max(t_count, f_count)
    
    t = pd.DataFrame(X[Y]).sample(c_max, replace=True).values
    f = pd.DataFrame(X[np.logical_not(Y)]).sample(c_max, replace=True).values
    
    X = np.vstack([t, f])
    Y = np.array([True] * c_max + [False] * c_max)
    
    return X, Y

In [52]:
%%time

tqdm.tqdm.monitor_interval = 0

validation_start = np.datetime64(cleaned_data["time"].max() - timedelta64(1, "Y"))
mccs = []

t = tqdm.tqdm(range(52), unit="week")
for i in t:
    cut = validation_start + i * timedelta64(7, "D")
    end = validation_start + (i + 1) * timedelta64(7, "D")
    
    train_mask = I < cut
    test_mask = np.logical_and(I >= cut, I < end)

    Xtrain = X[train_mask]
    Ytrain = Y[train_mask]
    
    Xtrain, Ytrain = balance(Xtrain, Ytrain)

    Xtest = X[test_mask]
    Ytest = Y[test_mask]
    
    ada = sklearn.ensemble.AdaBoostClassifier()
    ada.fit(Xtrain, Ytrain)
    Y_pred = ada.predict(Xtest)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mcc = sklearn.metrics.matthews_corrcoef(Ytest, Y_pred)
    mccs.append(mcc)
    t.set_postfix(mcc=mcc)

print(mean(mccs))





  0%|          | 0/52 [00:00<?, ?week/s]


  2%|▏         | 1/52 [00:07<06:43,  7.91s/week, mcc=0]


  4%|▍         | 2/52 [00:15<06:33,  7.86s/week, mcc=0]


  6%|▌         | 3/52 [00:23<06:25,  7.87s/week, mcc=0]


  8%|▊         | 4/52 [00:31<06:19,  7.91s/week, mcc=0]


 10%|▉         | 5/52 [00:39<06:13,  7.95s/week, mcc=0]


 12%|█▏        | 6/52 [00:47<06:07,  7.98s/week, mcc=0]


 13%|█▎        | 7/52 [00:56<06:10,  8.24s/week, mcc=0]


 15%|█▌        | 8/52 [01:05<06:07,  8.35s/week, mcc=0]


 17%|█▋        | 9/52 [01:13<06:02,  8.43s/week, mcc=0]


 19%|█▉        | 10/52 [01:22<06:02,  8.64s/week, mcc=0]


 21%|██        | 11/52 [01:32<06:02,  8.84s/week, mcc=0]


 23%|██▎       | 12/52 [01:40<05:51,  8.78s/week, mcc=0.597]


 25%|██▌       | 13/52 [01:49<05:38,  8.68s/week, mcc=0.565]


 27%|██▋       | 14/52 [01:57<05:28,  8.63s/week, mcc=0.739]


 29%|██▉       | 15/52 [02:06<05:18,  8.61s/week, mcc=0.647]


 31%|███       | 16/52 [02:15<05:12,  8.69s/week, mcc=0.607]



0.411555208351
CPU times: user 8min 12s, sys: 832 ms, total: 8min 13s
Wall time: 8min 17s


In [53]:
mcc[mcc > 0]

array([ 0.41155521])

In [56]:
mean([mcc for mcc in mccs if mcc > 0])

0.64522743728099041

In [57]:
max(mccs)

0.83484710993672195